<a href="https://colab.research.google.com/github/maryofcarmo/ICV/blob/main/MAPAS_CALOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1° Ligar ao Google e chamar bibliotecas necessárias

In [1]:
# conecta o notebook ao google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# inicia chamando todas as bibliotecas necessárias
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# 2° Ler o arquivo CSV já montado anteriomente, com todas as informações necessárias para plotagem

In [3]:
# ler o Arquivo com as informações das estações e coloca na variável est_info
# est_info é um dataframe
est_info = pd.read_csv('/content/drive/MyDrive/TCC/DADOS/INFORMACOES_ESTACOES/est_com_correlacao.csv')
est_info.head()

,nome do arquivo,Nome,Codigo Estacao,Latitude,Longitude,Altitude,Situacao,Data Inicial,Data Final,Periodicidade da Medicao,P1,P2,P3,K1,K2,K3,S1,S2,S3
0,Nome:_AGUA_BRANCA_Codigo_Estacao:_82989,AGUA BRANCA,82989,-9.264722,-37.938056,603.42,Operante,2000-01-01,2024-06-01,Mensal,0.789641,-0.771172,-0.952617,0.606061,-0.515152,-0.848485,0.804196,-0.713287,-0.937063
1,Nome:_ARAPIRACA_Codigo_Estacao:_82995,ARAPIRACA,82995,-9.730000,-36.770000,247.00,Desativada,2000-01-01,2024-06-01,Mensal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nome:_MACEIO_Codigo_Estacao:_82994,MACEIO,82994,-9.551389,-35.770833,84.12,Fechada,2000-01-01,2021-04-07,Mensal,0.668656,-0.659552,-0.492009,0.381691,-0.584615,-0.442761,0.570929,-0.805263,-0.647987
3,Nome:_MAJOR_ISIDORO_Codigo_Estacao:_82991,MAJOR ISIDORO,82991,-9.550000,-37.020000,200.00,Desativada,2000-01-01,2024-06-01,Mensal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nome:_MATA_GRANDE_Codigo_Estacao:_82988,MATA GRANDE,82988,-9.120000,-37.730000,635.00,Desativada,2000-01-01,2024-06-01,Mensal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Verificar quantas linhas têm pelo menos um valor vazio
linhas_com_vazio = est_info.isnull().any(axis=1).sum()
linhas_com_vazio

23

In [8]:
# retirar as linhas que tem pelo menos um valor vazio
est_info_limp = est_info.dropna()
est_info_limp

,nome do arquivo,Nome,Codigo Estacao,Latitude,Longitude,Altitude,Situacao,Data Inicial,Data Final,Periodicidade da Medicao,P1,P2,P3,K1,K2,K3,S1,S2,S3
0,Nome:_AGUA_BRANCA_Codigo_Estacao:_82989,AGUA BRANCA,82989,-9.264722,-37.938056,603.42,Operante,2000-01-01,2024-06-01,Mensal,0.789641,-0.771172,-0.952617,0.606061,-0.515152,-0.848485,0.804196,-0.713287,-0.937063
2,Nome:_MACEIO_Codigo_Estacao:_82994,MACEIO,82994,-9.551389,-35.770833,84.12,Fechada,2000-01-01,2021-04-07,Mensal,0.668656,-0.659552,-0.492009,0.381691,-0.584615,-0.442761,0.570929,-0.805263,-0.647987
5,Nome:_PALMEIRA_DOS_INDIOS_Codigo_Estacao:_82992,PALMEIRA DOS INDIOS,82992,-9.420278,-36.620278,277.98,Operante,2000-01-01,2024-06-01,Mensal,0.733587,-0.481364,-0.696482,0.484848,-0.212121,-0.606061,0.671329,-0.342657,-0.741259
6,Nome:_PAO_DE_ACUCAR_Codigo_Estacao:_82990,PAO DE ACUCAR,82990,-9.749167,-37.430833,20.86,Operante,2000-01-01,2024-06-01,Mensal,0.630290,-0.705835,-0.940163,0.454545,-0.636364,-0.818182,0.629371,-0.783217,-0.937063
8,Nome:_PORTO_DE_PEDRAS_Codigo_Estacao:_82996,PORTO DE PEDRAS,82996,-9.162012,-35.299881,42.21,Operante,2000-01-01,2024-06-01,Mensal,0.477049,-0.713409,-0.833801,0.181818,-0.515152,-0.545455,0.328671,-0.748252,-0.671329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Nome:_NATAL_Codigo_Estacao:_82598,NATAL,82598,-5.837222,-35.207778,47.68,Operante,2000-01-01,2024-06-01,Mensal,-0.928561,-0.778336,0.797946,-0.878788,-0.606061,0.606061,-0.951049,-0.790210,0.804196
112,Nome:_SERIDO_(CAICO)_Codigo_Estacao:_82690,SERIDO (CAICO),82690,-6.467500,-37.085000,171.26,Operante,2000-01-01,2024-06-01,Mensal,-0.463523,-0.057288,-0.304248,-0.212121,0.121212,-0.303030,-0.342657,0.048951,-0.440559
113,Nome:_ARACAJU_Codigo_Estacao:_83096,ARACAJU,83096,-10.952500,-37.054444,3.68,Fechada,2000-01-01,2020-12-31,Mensal,-0.884286,-0.910234,0.948704,-0.818182,-0.878788,0.818182,-0.944056,-0.972028,0.944056
114,Nome:_ITABAIANINHA_Codigo_Estacao:_83195,ITABAIANINHA,83195,-11.272222,-37.795000,205.59,Temporariamente Fechada,2000-01-01,2024-06-01,Mensal,0.731130,-0.612028,-0.885413,0.424242,-0.393939,-0.666667,0.615385,-0.587413,-0.825175
